#importing and getting the docker Image

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import json
import numpy as np
import os
import time

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

#Getting the im2rec.py to convert our images to RecordIO files

In [ ]:
%%bash
wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py
chmod +x im2rec.py

Creating .lst for our training and testing images

In [ ]:
%%bash
python im2rec.py --list --recursive  --test-ratio 0.3 --train-ratio 0.7 images AWS/MushroomLess/Data/Images/DATASet-Collected-by-MushroomLess/

#Generating the .Rec Files and meanwhile resizing them to "224" 

In [ ]:
%%bash
python im2rec.py --num-thread=4 --resize 224 images_train.lst AWS/MushroomLess/Data/Images/DATASet-Collected-by-MushroomLess/
python im2rec.py --num-thread=4 --resize 224 images_test.lst AWS/MushroomLess/Data/Images/DATASet-Collected-by-MushroomLess/

# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using

In [ ]:
bucket = sess.default_bucket()
s3train_path = 's3://{}/{}/train/'.format(MES3, MyFolder)
s3validation_path = 's3://{}/{}/validation/'.format(MES3, MyFolder)
# Uploading the rec files to the train and validation channels
!aws s3 cp /tmp/images_train.rec $s3train_path
!aws s3 cp /tmp/images_test.rec $s3validation_path

#Giving the S3Data_path for our ML - Channels

In [ ]:
train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

#Specifying the Output path for model.
and giving the Image-Docker
the needed configuration 

In [ ]:
output_path='s3://{}/{}/output/'.format(MES3, MyFolder)
image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p3.16xlarge',
    output_path=output_path,
    sagemaker_session=sess
)

#Specifying the hyperparameters for Our Model.

In [ ]:
#from train.lst
num_training_samples=
#from  # .lst
num_classes=
base_hyperparameters=dict(
    use_pretrained_model=1,
    image_shape='3,224,224',
    num_classes=num_classes,
    num_training_samples=num_training_samples,
)
hyperparameters={
    **base_hyperparameters, 
    **dict(
        learning_rate=0.001,
        mini_batch_size=10,
    )
}
image_classifier.set_hyperparameters(**hyperparameters)
hyperparameters

#Let the training start!

In [ ]:
%%time

now = str(int(time.time()))
training_job_name = 'IC-' +'MushroomLess-'+ now
image_classifier.fit(inputs=data_channels, job_name=training_job_name, logs=True)
job = image_classifier.latest_training_job

#Deploying the model to the Cloud

In [ ]:
%%time
deployed_endpoint = image_classifier.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.large'
)

In [ ]:
deployed_endpoint.endpoint

#Testing The Model

In [ ]:
import json
import numpy as np
import os

def classify_deployed(image_file, classes):
    payload = None
    with open(image_file, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)
        
    deployed_endpoint.content_type = 'application/x-image'
    result = json.loads(deployed_endpoint.predict(payload))
    best_prob_index = np.argmax(result)
    return (classes[best_prob_index], result[best_prob_index])